# Data Preparation

This parts aim to prepare the data before exploring it and building a prediction model. You must execute notebook `data_retrieval.ipynb` before executing this one.

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
from typing import List
import random

random.seed(42)

In [2]:
games = pd.read_csv('data/game_scores.csv', parse_dates=['date'])
stats = pd.read_csv('data/games_statistics.csv')
raw_data = games.merge(stats, on='boxscore_url', how='inner')
raw_data_backup = raw_data.copy()

### Filter All-Star game
We see the All-Star game is included, we want to discard it as it is not a regular game:

In [3]:
def get_teams(games_stats: pd.DataFrame) -> List[str]:
    """
    Given a game statistics dataframe, returns the unique list of teams.
    Take both home + away teams if the algorithm is run early in the season
    """
    return pd.concat([raw_data.home_team, raw_data.away_team], ignore_index=True).unique().tolist()

In [4]:
teams = get_teams(raw_data)
print(teams)
filtered_teams = ['All Star France', 'All Star Monde']
raw_data = raw_data[~raw_data.home_team.isin(filtered_teams)]
teams = get_teams(raw_data)
print(teams)

# Validation
assert all(team not in raw_data['home_team'] and team not in raw_data['away_team'] for team in filtered_teams)
assert all(team not in teams for team in filtered_teams)

['Dijon', 'Châlons-Reims', 'Boulogne-Levallois', 'Monaco', 'Chalon/Saône', 'Boulazac', 'Cholet', 'Bourg-en-Bresse', 'Lyon-Villeurbanne', 'Roanne', 'Pau-Lacq-Orthez', 'Limoges', 'Le Mans', 'Strasbourg', 'Nanterre', 'Le Portel', 'Gravelines-Dunkerque', 'Orléans', 'All Star France', 'All Star Monde']
['Dijon', 'Châlons-Reims', 'Boulogne-Levallois', 'Monaco', 'Chalon/Saône', 'Boulazac', 'Cholet', 'Bourg-en-Bresse', 'Lyon-Villeurbanne', 'Roanne', 'Pau-Lacq-Orthez', 'Limoges', 'Le Mans', 'Strasbourg', 'Nanterre', 'Le Portel', 'Gravelines-Dunkerque', 'Orléans']


### Win/Loss ratio computation

Compute team wins/loss before each game:

In [5]:
teams_WL_tmp = pd.DataFrame(data=np.zeros((len(teams), 2)), dtype=np.int64, columns=['wins', 'losses'], index=teams) # Creating a temporary dataframe to hold current team win/loss
raw_data.sort_values(by='date', axis='index', ascending=True, inplace=True) # Sorting by ascending dates
raw_data['home_team_wins'] = 0
raw_data['home_team_losses'] = 0
raw_data['away_team_wins'] = 0
raw_data['away_team_losses'] = 0

In [6]:
for index, row in raw_data.iterrows():
    raw_data.at[index, 'home_team_wins'] = teams_WL_tmp.loc[row["home_team"]]['wins']
    raw_data.at[index, 'away_team_wins'] = teams_WL_tmp.loc[row["away_team"]]['wins']
    raw_data.at[index, 'home_team_losses'] = teams_WL_tmp.loc[row["home_team"]]['losses']
    raw_data.at[index, 'away_team_losses'] = teams_WL_tmp.loc[row["away_team"]]['losses']
    
    if row['home_score'] > row['away_score']:
        teams_WL_tmp.at[row["home_team"], 'wins'] = teams_WL_tmp.at[row["home_team"], 'wins'] + 1
        teams_WL_tmp.at[row["away_team"], 'losses'] = teams_WL_tmp.at[row["away_team"], 'losses'] + 1
    elif row['home_score'] < row['away_score']:
        teams_WL_tmp.at[row["away_team"], 'wins'] = teams_WL_tmp.at[row["away_team"], 'wins'] + 1
        teams_WL_tmp.at[row["home_team"], 'losses'] = teams_WL_tmp.at[row["home_team"], 'losses'] + 1

raw_data['home_wining_percentage'] = raw_data['home_team_wins'] / (raw_data['home_team_wins'] + raw_data['home_team_losses'])
raw_data['away_wining_percentage'] = raw_data['away_team_wins'] / (raw_data['away_team_wins'] + raw_data['away_team_losses'])

# Validation
display(raw_data.sample(n=5, random_state=42))

,date,home_team,home_score,away_team,away_score,home_qt_1_pts,away_qt_1_pts,home_qt_2_pts,away_qt_2_pts,home_qt_3_pts,...,away_stl,away_tov,away_pf,away_pfd,home_team_wins,home_team_losses,away_team_wins,away_team_losses,home_wining_percentage,away_wining_percentage
220,2020-02-29,Nanterre,85,Boulazac,78,23,16,22,25,15,...,9,15,22,22,12,12,7,16,0.500000,0.304348
66,2019-11-08,Le Portel,98,Boulogne-Levallois,79,32,24,24,7,18,...,14,9,23,20,1,7,8,0,0.125000,1.000000
9,2019-09-25,Lyon-Villeurbanne,78,Boulazac,69,18,19,23,13,18,...,3,14,21,21,1,0,1,0,1.000000,1.000000
171,2020-01-19,Lyon-Villeurbanne,104,Bourg-en-Bresse,71,24,30,36,10,23,...,6,19,34,26,16,2,12,6,0.888889,0.666667
13,2019-09-28,Limoges,66,Le Portel,65,23,27,18,15,14,...,7,13,24,21,0,1,0,1,0.000000,0.000000


### Possessions & Pace

Computation of the number of possessions. We used formula $FGA + 0.44*FTA - ORB + TOV$. See notes for more in-depth information. We can then infer the pace (possessions per 40 minutes), by computing $\dfrac{40 * possessions}{minutes}$ to see how fast a team plays.  
(*Note that `minutes` column is divided by 5, as it is the total played, by all 5 players on the floor.*).

In [7]:
raw_data['home_possessions'] = round(raw_data['home_2pa'] + raw_data['home_3pa'] + 0.44 * raw_data['home_fta'] - raw_data['home_orbd'] + raw_data['home_tov'], 2)
raw_data['away_possessions'] = round(raw_data['away_2pa'] + raw_data['away_3pa'] + 0.44 * raw_data['away_fta'] - raw_data['away_orbd'] + raw_data['away_tov'], 2)
raw_data['home_pace'] = round((raw_data['home_possessions']*40)/(raw_data['minutes']/5), 2)
raw_data['away_pace'] = round((raw_data['away_possessions']*40)/(raw_data['minutes']/5), 2)

assert all(np.greater(raw_data[raw_data.minutes > 200].home_possessions, raw_data[raw_data.minutes > 200].home_pace))
assert all(np.equal(raw_data[raw_data.minutes == 200].home_pace, raw_data[raw_data.minutes == 200].home_possessions))
assert all(np.greater(raw_data[raw_data.minutes > 200].away_possessions, raw_data[raw_data.minutes > 200].away_pace))
assert all(np.equal(raw_data[raw_data.minutes == 200].away_pace, raw_data[raw_data.minutes == 200].away_possessions))

display(raw_data[['home_team', 'away_team', 'home_possessions', 'away_possessions', 'home_pace', 'away_pace']].sample(n=5, random_state=42))

,home_team,away_team,home_possessions,away_possessions,home_pace,away_pace
220,Nanterre,Boulazac,75.68,73.20,75.68,73.20
66,Le Portel,Boulogne-Levallois,83.44,84.00,83.44,84.00
9,Lyon-Villeurbanne,Boulazac,72.84,70.32,72.84,70.32
171,Lyon-Villeurbanne,Bourg-en-Bresse,72.28,73.96,72.28,73.96
13,Limoges,Le Portel,75.00,73.28,75.00,73.28


### Offensive Rating (ORtg), Defensive Rating (DRtg) and Net rating (NRtg)

Now that we have the pace, we can derive Offensive Rating, Defensive Rating and Net Rating. Offensive Rating is points scored per 100 possessions:  
$\dfrac{Pts * 100}{Poss}$  
  
Defensive rating is basically the opponent's offensive rating, and finally Net Rating is:  
$NRtg = ORtg - DRtg$.

In [8]:
raw_data['home_ortg'] = raw_data['home_score'] * 100 / raw_data['home_possessions']
raw_data['away_ortg'] = raw_data['away_score'] * 100 / raw_data['away_possessions']
raw_data['home_drtg'] = raw_data['away_ortg']
raw_data['away_drtg'] = raw_data['home_ortg']
raw_data['home_nrtg'] = raw_data['home_ortg'] - raw_data['home_drtg']
raw_data['away_nrtg'] = raw_data['away_ortg'] - raw_data['away_drtg']

display(raw_data[[
    'home_team', 
    'away_team', 
    'home_score', 
    'away_score', 
    'home_possessions', 
    'away_possessions',
    'home_ortg',
    'away_ortg',
    'home_drtg',
    'away_drtg',
    'home_nrtg',
    'away_nrtg',
]].sample(n=5, random_state=42))

,home_team,away_team,home_score,away_score,home_possessions,away_possessions,home_ortg,away_ortg,home_drtg,away_drtg,home_nrtg,away_nrtg
220,Nanterre,Boulazac,85,78,75.68,73.20,112.315011,106.557377,106.557377,112.315011,5.757634,-5.757634
66,Le Portel,Boulogne-Levallois,98,79,83.44,84.00,117.449664,94.047619,94.047619,117.449664,23.402045,-23.402045
9,Lyon-Villeurbanne,Boulazac,78,69,72.84,70.32,107.084020,98.122867,98.122867,107.084020,8.961153,-8.961153
171,Lyon-Villeurbanne,Bourg-en-Bresse,104,71,72.28,73.96,143.884892,95.997837,95.997837,143.884892,47.887055,-47.887055
13,Limoges,Le Portel,66,65,75.00,73.28,88.000000,88.700873,88.700873,88.000000,-0.700873,0.700873


### Compute average team statistics before each game

Pseudo code of the function to compute a stat for all previous game efficiently:
```
acc_dataframe <- Create dataframe [team=list of unique teams, sum_stats=np.Nan, nb_games_played=0]
raw_data <- Sort raw_data by game date

for game in raw_data:
    if first game of team:
        game[home_stat] <- NaN
    else:
        game[home_stat] <- acc_dataframe[game[home_team], sum_stats] / acc_dataframe[game[home_team], nb_games_played]
        
    acc_dataframe[team][sum_statistic] += game[home_stat]
    acc_dataframe[team][nb_game_played] += 1
```

In [9]:
def avg_stat(games_stats: pd.DataFrame, statistic: str):
    """
    Given a game statistics dataframe and a statistic category (rbd, blk, ast, etc.) 
    computes the average for each team (home, away) before each game.
    """
    teams = get_teams(games_stats)
    acc_dataframe = pd.DataFrame(data=np.zeros((len(teams), 2)), columns=['sum_' + statistic, 'games_played'], index=teams)
    raw_data.sort_values(by='date', axis='index', ascending=True, inplace=True) # Sorting by ascending dates
    
    for idx, game in games_stats.iterrows():
        for team in ['home', 'away']:
            # Computing average for the team
            if acc_dataframe.loc[game[team + '_team'], 'games_played'] == 0: # First game
                games_stats.loc[idx, team + '_avg_' + statistic] = np.nan
            else: # All other games
                games_stats.loc[idx, team + '_avg_' + statistic] = acc_dataframe.loc[game[team + '_team'], 'sum_' + statistic] / acc_dataframe.loc[game[team + '_team'], 'games_played']
    
            # Adding current game stats to the team accumulated stats.
            acc_dataframe.loc[game[team + '_team'], 'sum_' + statistic] += games_stats.loc[idx, team + '_' + statistic]
            acc_dataframe.loc[game[team + '_team'], 'games_played'] += 1

In [10]:
raw_data.columns

Index(['date', 'home_team', 'home_score', 'away_team', 'away_score',
       'home_qt_1_pts', 'away_qt_1_pts', 'home_qt_2_pts', 'away_qt_2_pts',
       'home_qt_3_pts', 'away_qt_3_pts', 'home_qt_4_pts', 'away_qt_4_pts',
       'home_ot_1_pts', 'away_ot_1_pts', 'home_ot_2_pts', 'away_ot_2_pts',
       'home_ot_3_pts', 'away_ot_3_pts', 'boxscore_url', 'minutes', 'home_ast',
       'home_drbd', 'home_orbd', 'home_2pa', 'home_2pm', 'home_3pa',
       'home_3pm', 'home_fta', 'home_ftm', 'home_blk', 'home_stl', 'home_tov',
       'home_pf', 'home_pfd', 'away_ast', 'away_drbd', 'away_orbd', 'away_2pa',
       'away_2pm', 'away_3pa', 'away_3pm', 'away_fta', 'away_ftm', 'away_blk',
       'away_stl', 'away_tov', 'away_pf', 'away_pfd', 'home_team_wins',
       'home_team_losses', 'away_team_wins', 'away_team_losses',
       'home_wining_percentage', 'away_wining_percentage', 'home_possessions',
       'away_possessions', 'home_pace', 'away_pace', 'home_ortg', 'away_ortg',
       'home_drtg', 'awa

In [11]:
avg_stat(raw_data, 'score')
raw_data = raw_data.rename({'home_avg_score': 'home_avg_pts', 'away_avg_score': 'away_avg_pts'})
avg_stat(raw_data, 'qt_1_pts')
avg_stat(raw_data, 'qt_2_pts')
avg_stat(raw_data, 'qt_3_pts')
avg_stat(raw_data, 'qt_4_pts')
avg_stat(raw_data, 'ast')
avg_stat(raw_data, 'drbd')
avg_stat(raw_data, 'orbd')
avg_stat(raw_data, '2pm')
avg_stat(raw_data, '2pa')
avg_stat(raw_data, '3pm')
avg_stat(raw_data, '3pa')
avg_stat(raw_data, 'ftm')
avg_stat(raw_data, 'fta')
avg_stat(raw_data, 'blk')
avg_stat(raw_data, 'stl')
avg_stat(raw_data, 'tov')
avg_stat(raw_data, 'pf')
avg_stat(raw_data, 'pfd')
avg_stat(raw_data, 'possessions')
avg_stat(raw_data, 'pace')
avg_stat(raw_data, 'ortg')
avg_stat(raw_data, 'drtg')
avg_stat(raw_data, 'nrtg')
display(raw_data.iloc[random.randint(0, raw_data.shape[0])])

date             2020-01-17 00:00:00
home_team              Châlons-Reims
home_score                        78
away_team                    Le Mans
away_score                        87
                        ...         
away_avg_ortg                 112.65
home_avg_drtg                111.353
away_avg_drtg                114.541
home_avg_nrtg                1.00684
away_avg_nrtg               -1.89176
Name: 164, Length: 113, dtype: object

### Adding budgets and salary masses

In [12]:
budgets = pd.read_csv('data/budgets.csv')
budgets = budgets[budgets['season'] == '2019-2020']
budgets = budgets.drop(columns=['season', 'rank', 'rank.1'])
budgets.head()

,team,budget,salary_mass
0,Boulazac,3794000,1509000
1,Bourg-en-Bresse,5354000,1702000
2,Châlons-Reims,4253000,1406000
3,Chalon/Saône,5452000,1736000
4,Cholet,4601000,1365000


In [13]:
raw_data = pd.merge(raw_data, budgets, left_on='home_team', right_on='team', how='inner') \
    .drop(columns=['team']) \
    .rename({'budget': 'home_team_budget', 'salary_mass': 'home_team_salary_mass'}, axis='columns')

raw_data = pd.merge(raw_data, budgets, left_on='away_team', right_on='team', how='inner') \
    .drop(columns=['team']) \
    .rename({'budget': 'away_team_budget', 'salary_mass': 'away_team_salary_mass'}, axis='columns')

raw_data.head()

,date,home_team,home_score,away_team,away_score,home_qt_1_pts,away_qt_1_pts,home_qt_2_pts,away_qt_2_pts,home_qt_3_pts,...,home_avg_ortg,away_avg_ortg,home_avg_drtg,away_avg_drtg,home_avg_nrtg,away_avg_nrtg,home_team_budget,home_team_salary_mass,away_team_budget,away_team_salary_mass
0,2019-09-20,Dijon,88,Le Portel,85,26,19,21,28,17,...,NaN,NaN,NaN,NaN,NaN,NaN,4967000,1625000,3948000,1250000
1,2019-12-23,Châlons-Reims,97,Le Portel,87,29,16,20,21,28,...,110.145148,101.318795,108.849069,115.081726,1.296078,-13.762931,4253000,1406000,3948000,1250000
2,2019-12-14,Chalon/Saône,95,Le Portel,84,24,26,30,22,20,...,103.427595,102.322540,116.183755,114.869922,-12.756160,-12.547383,5452000,1736000,3948000,1250000
3,2020-01-18,Boulazac,80,Le Portel,74,17,18,27,19,15,...,108.964936,103.154736,113.418723,116.025048,-4.453786,-12.870312,3794000,1509000,3948000,1250000
4,2019-10-26,Cholet,80,Le Portel,72,21,23,17,14,22,...,109.970408,100.495657,104.030839,108.179698,5.939569,-7.684041,4601000,1365000,3948000,1250000


### Adding classification target variable

In [14]:
raw_data['home_team_win'] = raw_data['home_score'] > raw_data['away_score']
raw_data.sample(n=5)

,date,home_team,home_score,away_team,away_score,home_qt_1_pts,away_qt_1_pts,home_qt_2_pts,away_qt_2_pts,home_qt_3_pts,...,away_avg_ortg,home_avg_drtg,away_avg_drtg,home_avg_nrtg,away_avg_nrtg,home_team_budget,home_team_salary_mass,away_team_budget,away_team_salary_mass,home_team_win
133,2019-09-28,Pau-Lacq-Orthez,87,Bourg-en-Bresse,93,17,26,21,24,22,...,116.009281,109.325606,104.772992,2.519844,11.236289,5648000,1620000,5354000,1702000,False
140,2019-09-20,Boulogne-Levallois,93,Orléans,87,28,27,22,17,17,...,NaN,NaN,NaN,NaN,NaN,6755000,2285000,4504000,1531500,True
61,2019-12-20,Le Portel,60,Monaco,83,10,18,11,26,20,...,118.946274,115.413915,100.115742,-12.587628,18.830532,3948000,1250000,9056000,3595000,False
123,2019-12-02,Gravelines-Dunkerque,73,Cholet,79,19,18,17,13,20,...,111.830199,111.046719,106.237151,-0.989805,5.593048,5971000,2066500,4601000,1365000,False
134,2020-02-29,Le Mans,91,Bourg-en-Bresse,93,16,26,31,20,24,...,112.100304,113.102212,110.830802,-1.240759,1.269502,6011000,1852000,5354000,1702000,False


### Removing/Replacing *Null* values

We will modify overtime `NaN` values into $0$. On the other hand, as we don't have team averages on the first game of each season, we will remove those rows.

In [15]:
raw_data.columns[raw_data.isna().any()]

Index(['home_ot_1_pts', 'away_ot_1_pts', 'home_ot_2_pts', 'away_ot_2_pts',
       'home_ot_3_pts', 'away_ot_3_pts', 'home_wining_percentage',
       'away_wining_percentage', 'home_avg_score', 'away_avg_score',
       'home_avg_qt_1_pts', 'away_avg_qt_1_pts', 'home_avg_qt_2_pts',
       'away_avg_qt_2_pts', 'home_avg_qt_3_pts', 'away_avg_qt_3_pts',
       'home_avg_qt_4_pts', 'away_avg_qt_4_pts', 'home_avg_ast',
       'away_avg_ast', 'home_avg_drbd', 'away_avg_drbd', 'home_avg_orbd',
       'away_avg_orbd', 'home_avg_2pm', 'away_avg_2pm', 'home_avg_2pa',
       'away_avg_2pa', 'home_avg_3pm', 'away_avg_3pm', 'home_avg_3pa',
       'away_avg_3pa', 'home_avg_ftm', 'away_avg_ftm', 'home_avg_fta',
       'away_avg_fta', 'home_avg_blk', 'away_avg_blk', 'home_avg_stl',
       'away_avg_stl', 'home_avg_tov', 'away_avg_tov', 'home_avg_pf',
       'away_avg_pf', 'home_avg_pfd', 'away_avg_pfd', 'home_avg_possessions',
       'away_avg_possessions', 'home_avg_pace', 'away_avg_pace',
       'home

In [16]:
# Dropping all rows which either the home or away team has nto an average score (first game of season for the team).
raw_data = raw_data.dropna(axis='index', subset=['home_avg_score', 'away_avg_score'])
display(raw_data.columns[raw_data.isna().any()])

# Filling remaining NaN values (overtimes) to 0
raw_data = raw_data.fillna(0)
assert len(raw_data.columns[raw_data.isna().any()]) == 0

Index(['home_ot_1_pts', 'away_ot_1_pts', 'home_ot_2_pts', 'away_ot_2_pts',
       'home_ot_3_pts', 'away_ot_3_pts'],
      dtype='object')

### Removing current game data

Our models will take data known *before* a game in input, to predict ifthe home team will win or not. Thus we can remove all statistics of the current game of the dataset.

In [17]:
list(raw_data.columns)

['date',
 'home_team',
 'home_score',
 'away_team',
 'away_score',
 'home_qt_1_pts',
 'away_qt_1_pts',
 'home_qt_2_pts',
 'away_qt_2_pts',
 'home_qt_3_pts',
 'away_qt_3_pts',
 'home_qt_4_pts',
 'away_qt_4_pts',
 'home_ot_1_pts',
 'away_ot_1_pts',
 'home_ot_2_pts',
 'away_ot_2_pts',
 'home_ot_3_pts',
 'away_ot_3_pts',
 'boxscore_url',
 'minutes',
 'home_ast',
 'home_drbd',
 'home_orbd',
 'home_2pa',
 'home_2pm',
 'home_3pa',
 'home_3pm',
 'home_fta',
 'home_ftm',
 'home_blk',
 'home_stl',
 'home_tov',
 'home_pf',
 'home_pfd',
 'away_ast',
 'away_drbd',
 'away_orbd',
 'away_2pa',
 'away_2pm',
 'away_3pa',
 'away_3pm',
 'away_fta',
 'away_ftm',
 'away_blk',
 'away_stl',
 'away_tov',
 'away_pf',
 'away_pfd',
 'home_team_wins',
 'home_team_losses',
 'away_team_wins',
 'away_team_losses',
 'home_wining_percentage',
 'away_wining_percentage',
 'home_possessions',
 'away_possessions',
 'home_pace',
 'away_pace',
 'home_ortg',
 'away_ortg',
 'home_drtg',
 'away_drtg',
 'home_nrtg',
 'away_nrtg'

In [18]:
raw_data = raw_data.drop(columns=[
    'home_score',
    'away_score',
    'home_qt_1_pts',
    'away_qt_1_pts',
    'home_qt_2_pts',
    'away_qt_2_pts',
    'home_qt_3_pts',
    'away_qt_3_pts',
    'home_qt_4_pts',
    'away_qt_4_pts',
    'home_ot_1_pts',
    'away_ot_1_pts',
    'home_ot_2_pts',
    'away_ot_2_pts',
    'home_ot_3_pts',
    'away_ot_3_pts',
    'boxscore_url',
    'minutes',
    'home_ast',
    'home_drbd',
    'home_orbd',
    'home_2pa',
    'home_2pm',
    'home_3pa',
    'home_3pm',
    'home_fta',
    'home_ftm',
    'home_blk',
    'home_stl',
    'home_tov',
    'home_pf',
    'home_pfd',
    'away_ast',
    'away_drbd',
    'away_orbd',
    'away_2pa',
    'away_2pm',
    'away_3pa',
    'away_3pm',
    'away_fta',
    'away_ftm',
    'away_blk',
    'away_stl',
    'away_tov',
    'away_pf',
    'away_pfd',
    'home_possessions',
    'away_possessions',
    'home_pace',
    'away_pace',
    'home_ortg',
    'away_ortg',
    'home_drtg',
    'away_drtg',
    'home_nrtg',
    'away_nrtg'
])

### Saving processed data

In [19]:
raw_data.to_csv('data/processed_data.csv', index=False)